In [1]:
import psycopg2
import csv
import utils
import re
import census
from collections import defaultdict
import dill

# Set-up: Connect to the database
Two databases available:
* census_2016_04_spider_4
* census_2016_08_25k_id_detection_1

This cell will set the 'con' object — the database connection. It gets passed to all census functions.

In [2]:
dbname = 'census_2016_08_25k_id_detection_1' # Change me!

user = 'openwpm'
password = 'Pwcpdtwca!'
host = 'princeton-web-census-machine-1.cp85stjatkdd.us-east-1.rds.amazonaws.com' 
db_details = 'dbname={0} user={1} password={2} host={3}'.format(dbname, user, password, host)


con = psycopg2.connect(db_details)

# Available API

## Check to see that a given top_url is present in the dataset
All data in the census is keyed by each 'top_url' visited. Each top_url follows the format:  

`http://example.com`  

There is never a leading '`www.`', nor is the scheme ever '`https://`'. If a site redirects to https, that will be reflected in the crawl's data.

In [3]:
top_url = 'http://netflix.com'
print(census.check_top_url(con, top_url))

top_url = 'http://notincensus.com'
print(census.check_top_url(con, top_url))


True
False


## Get all third party responses by top_url
`census.get_third_party_responses_by_domain(con, top_url)` returns a two-level results dict containing third party urls loaded on the given top_url.

The dict's structure is:

`dict[third_party_url]['is_tracker']`, contains True if third_party_url is identified on a blocklist.  
`dict[third_party_url]['is_js']`, contains True if third_party_url is a script.  
`dict[third_party_url]['is_img']`, contains True if third_party_url is an image.  
`dict[third_party_url]['url_ps']`, contains the string for the public-suffix+1 of the url.  



In [4]:
top_url = 'http://espn.go.com'

results = census.get_third_party_responses_by_domain(con, 'http://espn.go.com')

third_party_trackers = {results[x]['url_ps'] for x in results if results[x]['is_tracker']}

print("Number of third_party trackers on domain: " + str(len(third_party_trackers)))

for url in results:
    print(url)
    print('\tIs a script? ' + str(results[url]['is_js']))
    print('\tIs a tracker? ' + str(results[url]['is_tracker']))
    print('\tPS+1: ' + results[url]['url_ps'])

Number of third_party trackers on domain: 16
http://pagead2.googlesyndication.com/activeview?avi=B9gIMj1a7V-CdHc6V3AHY5aW4DQAAAAAQATgByAEJwAIC4AIA4AQBoAYW&cid=CAASFeRoP0gZLoy42SA-n3lfDpS1dwKbzQ&id=osdim&ti=1&adk=1520060612&p=1,0,1,0&tos=0,0,0,0,0&mtos=0,0,0,0,0&rs=3&ht=0&mc=0&lte=-1&bas=0&bac=0&fp=correlator%3D3524041284612949%26eid%3D108809080%252C108809112%26iu%3D%252F6444%252Fespn.com%252Ffrontpage%252Findex%26oid%3D3%26url%3Dhttp%253A%252F%252Fwww.espn.com%252F&afp=%26output%3Djson_html%26impl%3Dfif%26dt%3D1471895182957%26adx%3D35%26ady%3D128%26ifi%3D7%26flash%3D11.2.202&r=u&bs=1350,697&bos=1366,768&ps=1350,10639&ss=1366,768&tt=16471&pt=-1&deb=1-0-3-25-18--1&tvt=9101&uc=1&tgt=nf&cl=0
	Is a script? False
	Is a tracker? True
	PS+1: googlesyndication.com
http://a.espncdn.com/combiner/i?img=/i/teamlogos/countries/500/fra.png&h=20&w=20
	Is a script? False
	Is a tracker? False
	PS+1: espncdn.com
http://a1.espncdn.com/combiner/i?img=/i/teamlogos/soccer/500/94.png&h=20&w=20
	Is a script? F

## Get all third party responses for a list of sites
A cell to simplify getting all third party responses for a given list of sites.

In [5]:
sites = ['http://cnn.com', 'http://wsj.com'] # Change me!

# Resulting dictionaries
tracker_js_by_top = defaultdict(set)
tracker_img_by_top = defaultdict(set)
non_tracker_js_by_top = defaultdict(set)
non_tracker_img_by_top = defaultdict(set)

tracker_other_by_top = defaultdict(set)
non_tracker_other_by_top = defaultdict(set)
for site in sites:
    tp_data = census.get_third_party_responses_by_domain(con, site)
    for url in tp_data:
        url_ps = tp_data[url]['url_ps']
        is_tracker = tp_data[url]['is_tracker']
        if is_tracker:
            if tp_data[url]['is_js']:
                tracker_js_by_top[site].add(url_ps)
            elif tp_data[url]['is_img']:
                tracker_img_by_top[site].add(url_ps)
            else:
                tracker_other_by_top[site].add(url_ps)
        else:
            if tp_data[url]['is_js']:
                non_tracker_js_by_top[site].add(url_ps)
            elif tp_data[url]['is_img']:
                non_tracker_img_by_top[site].add(url_ps)           
            else:
                non_tracker_other_by_top[site].add(url_ps)


In [7]:
# Save output as .dill

with open('tracker_js_by_top.dill', 'wb') as f:
    dill.dump(tracker_js_by_top, f)
with open('tracker_js_by_top.dill', 'wb') as f:
    dill.dump(non_tracker_js_by_top, f)
with open('tracker_img_by_top.dill', 'wb') as f:
    dill.dump(tracker_img_by_top, f)
with open('non_tracker_img_by_top.dill', 'wb') as f:
    dill.dump(non_tracker_img_by_top, f)
with open('tracker_other_by_top.dill', 'wb') as f:
    dill.dump(tracker_other_by_top, f)
with open('non_tracker_other_by_top.dill', 'wb') as f:
    dill.dump(non_tracker_other_by_top, f)

In [8]:
# Save output in CSVs
with open('tracker_js_by_top.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['top_url', 'tp_domain'])
    for top in tracker_js_by_top:
        for tp in tracker_js_by_top[top]:
            writer.writerow([top, tp])
with open('non_tracker_js_by_top.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['top_url', 'tp_domain'])
    for top in non_tracker_js_by_top:
        for tp in non_tracker_js_by_top[top]:
            writer.writerow([top, tp])
with open('tracker_img_by_top.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['top_url', 'tp_domain'])
    for top in tracker_img_by_top:
        for tp in tracker_img_by_top[top]:
            writer.writerow([top, tp])
with open('non_tracker_img_by_top.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['top_url', 'tp_domain'])
    for top in non_tracker_img_by_top:
        for tp in non_tracker_img_by_top[top]:
            writer.writerow([top, tp])
with open('tracker_other_by_top.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['top_url', 'tp_domain'])
    for top in tracker_other_by_top:
        for tp in tracker_other_by_top[top]:
            writer.writerow([top, tp])
with open('non_tracker_other_by_top.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['top_url', 'tp_domain'])
    for top in non_tracker_other_by_top:
        for tp in non_tracker_other_by_top[top]:
            writer.writerow([top, tp])

## Get top_urls that load a resource from a given third party domain

`census.get_top_urls_with_third_party_domain(con, tp_domain)` returns a dictionary mapping top_urls in the census to a list of urls that were loaded on that top_url belonging to a certain tp_domain.

In [9]:
tp_domain = 'addthis.com'

tps_by_top = census.get_top_urls_with_third_party_domain(con, tp_domain)

print("Number of top_urls with given third party : " + str(len(tps_by_top)))

Number of top_urls with given third party : 1299


## Get "cookie sync" events on a given top_url
Note: This does not include logic for isolating "identifying cookies." Any cookies of a sufficient cookie length that are shared with other domains will be identified.

In [10]:
# For a single top_url...

results = census.get_cookie_syncs_v2(con, 'http://microsoft.com', cookie_length=8)

for receiving_url in results:
    print("R: " + receiving_url)
    for sending_url, val in results[receiving_url]:
        print("\tS: " + sending_url)
        print("\tV: " + val)

R: http://ib.adnxs.com/getuid?https%3A%2F%2Ftags.bluekai.com%2Fsite%2F3085%3Fid%3D%24UID%26BK_SWAP_DEST%3D3085
	S: bluekai.com
	V: 2248F39DA07162F82B6BFAE4A19D63F9
R: http://googleads.g.doubleclick.net/pagead/viewthroughconversion/1038632207/?value=1.00&label=b4o6CJ7w5lYQj4qh7wM&guid=ON&script=0
	S: bluekai.com
	V: 2248F39DA07162F82B6BFAE4A19D63F9
R: https://6212760188.log.optimizely.com/event?a=6212760188&d=3698060313&y=true&src=js&x6600010229=6919882420&s6202010951=direct&s6206680296=ff&s6183560892=none&s6208020262=false&tsent=1471881874.32&n=http%3A%2F%2Fwww.microsoft.com%2Fen-us%2F&u=oeu1471881874167r0.7410621262857278&wxhr=true&time=1471881874.319&f=6600010229&g=6600010229&cx2=a8699c41
	S: microsoft.com
	V: oeu1471881874167r0.7410621262857278
R: http://d.btttag.com/btt.gif?pageName=MS_com%20Home%20Page&nst=1471881870165&unloadEventStart=0&redirectStart=0&totRD=0&fetchStart=1471881870467&domainLookupStart=1471881870467&connectStart=1471881870467&secureConnectionStart=0&requestStart

In [ ]:
# For a list of top_urls...
# Warning: this method is slow.

sites = ['http://microsoft.com', 'http://cnn.com']  # Change me!

cookie_sync_data = defaultdict(defaultdict)
for i, site in sites:
    print(site)
    cookie_sync_data[site] = census.get_cookie_syncs_v2(con, site, cookie_length=8)


In [ ]:
# Write output as .dill
with open('cookie_syncs.dill', 'w') as f:
    dill.dump(cookie_sync_data, f)

# Write complete output as csv
with open('cookie_syncs.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['top_url', 'sending_domain', 'receiving_url', 'cookie_value'])
    for site in cookie_sync_data:
        for receiving_url in cookie_sync_data[site]:
            for sending_url, cookie_value in cookie_sync_data[site][receiving_url]:
                writer.writerow([site, sending_url, receiving_url, cookie_value])

# Write partial output as CSV, only identifying sending domain and receiving domain
# (rather than the full receiving URL)

cooks_just_domains = defaultdict(defaultdict)
for site in cookie_sync_data:
    cooks_just_domains[site] = defaultdict(set)
    for receiving_url in cookie_sync_data[site]:
        for sending_domain, value in cookie_sync_data[site][receiving_url]:
            cooks_just_domains[site][utils.get_host_plus_ps(receiving_url)].add(sending_domain)
with open('../wsj/cookie_syncs_v2_just_domains.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['top_url', 'sending_domain', 'receiving_domain'])
    for site in cooks_just_domains:
        for receiving_domain in cooks_just_domains[site]:
            if len(cooks_just_domains[site][receiving_domain]) > 1 and 'NOT_FOUND' in cooks_just_domains[site][receiving_domain]:
                cooks_just_domains[site][receiving_domain].discard('NOT_FOUND')
            for sending_domain in cooks_just_domains[site][receiving_domain]:
                writer.writerow([site, sending_domain, receiving_domain])

## Check a given url against a blocklist
Available blocklists:
* easylist.txt
* easyprivacy.txt

In [ ]:
print(utils.is_tracker('http://tags.bkrtx.com/js/bk-coretag.js', is_js=True, is_img=False, 
                       first_party='http://verizonwireless.com', blocklist='easylist.txt'))


## Get third party scripts on given top_url that call particular javascript symbol

In [ ]:
print(census.get_urls_with(con, 'http://cnn.com', 'CanvasRenderingContext2D.fillText'))

## Get the PS+1 of a given top_url

In [ ]:
print(utils.get_host_plus_ps('http://subdomain.example.com/this/will/be/deleted.jpg'))